In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model = "gpt-4o-mini"

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

loader = WebBaseLoader('https://www.nytimes.com/')
documents = loader.load_and_split(text_splitter)
documents

[Document(metadata={'source': 'https://www.nytimes.com/', 'title': 'The New York Times - Breaking News, US News, World News and Videos', 'description': 'Live news, investigations, opinion, photos and video by the journalists of The New York Times from more than 150 countries around the world. Subscribe for coverage of U.S. and international news, politics, business, technology, science, health, arts, sports and more.', 'language': 'en'}, page_content='The New York Times - Breaking News, US News, World News and Videos'),
 Document(metadata={'source': 'https://www.nytimes.com/', 'title': 'The New York Times - Breaking News, US News, World News and Videos', 'description': 'Live news, investigations, opinion, photos and video by the journalists of The New York Times from more than 150 countries around the world. Subscribe for coverage of U.S. and international news, politics, business, technology, science, health, arts, sports and more.', 'language': 'en'}, page_content='Skip to contentSki

In [10]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    documents, embedding=OpenAIEmbeddings()
)

In [11]:
import pandas as pd

df = pd.DataFrame([d.page_content for d in documents], columns=["text"])
df.head(10)

,text
0,"The New York Times - Breaking News, US News, W..."
1,Skip to contentSkip to site indexSKIP ADVERTIS...
2,Global UpdateThe latest news for any part of t...
3,"music news, new songs and albums, and artists ..."
4,Ezra Klein ShowDiscussions of ideas that matte...
5,Bee ForumWordplay ColumnWordle ReviewSubmit a ...
6,included in an All Access subscription. Learn ...
7,"in baseball, by Levi Weaver with Ken Rosenthal..."
8,EuropeGet what you need to know to start your ...
9,TV and movie recommendations.See all newslette...


In [12]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(df)

In [13]:
from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base,
    num_questions=60,
    agent_description="A bot answering questions about Lifo Website",
)

2024-08-27 11:18:52,148 pid:19128 MainThread giskard.rag  INFO     Finding topics in the knowledge base.


c:\Users\c.manara\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\umap\umap_.py:2437: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


2024-08-27 11:18:53,705 pid:19128 MainThread giskard.rag  INFO     Found 1 topics in the knowledge base.


Generating questions: 100%|██████████| 60/60 [08:18<00:00,  8.31s/it]


In [16]:
df = testset.to_pandas()

for index, row in enumerate(df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")

Question 1: What is the concern about the transition to electric cars in South Korea?
Reference answer: A dramatic fire in an underground lot involving a Mercedes-Benz electric vehicle has left potential buyers in South Korea worried about safety.
Reference context:
Document 21: the court considers the case’s merits.4 min readA Mercedes-Benz Fire Jolts South Korea’s E.V. TransitionThe government is aggressively promoting electric cars. But a dramatic blaze in an underground lot has left some potential buyers worried about safety.4 min readRahmat Khan/Associated PressWave of Attacks Rattles a Restive Province in PakistanAt least 38 people were killed in several assaults across Baluchistan Province since Sunday in what appears to be part of a campaign by armed separatists.5 min readThe AthleticSports coverageAdvertisementSKIP ADVERTISEMENTWellCulture and LifestyleCongratulations! You Made the Film Festival. Now Finish Your Movie.With the Venice Film Festival beginning, filmmakers are rac

In [26]:
testset.save("test-set.jsonl")

In [17]:
from langchain.prompts import PromptTemplate

template = """
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise. \
       
Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.        
Context: Here is some context

Question: Here is a question



In [18]:
retriever = vectorstore.as_retriever()
retriever.get_relevant_documents("What about the Russia-Ukraine war?")

c:\Users\c.manara\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'source': 'https://www.nytimes.com/', 'title': 'The New York Times - Breaking News, US News, World News and Videos', 'description': 'Live news, investigations, opinion, photos and video by the journalists of The New York Times from more than 150 countries around the world. Subscribe for coverage of U.S. and international news, politics, business, technology, science, health, arts, sports and more.', 'language': 'en'}, page_content='Russia-Ukraine WarPhotosMap: Ukraine’s IncursionUkraine’s Mine-Detecting DronesA Poetry Boom in UkraineIn Eastern Ukraine, Terrifying Bombardment and Near Total DestructionPowerful guided “glide bombs” have helped Russia raze towns with greater speed. “When you drive into a ruined town, it’s like hopelessness,” one soldier said.5 min readTyler Hicks/The New York TimesAssassination Task Force Kicks Off as Hard-Right Talks of ‘Parallel’ InquiryAs the bipartisan group investigating the assassination attempt on Donald Trump visited Pennsylvan

In [20]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

model_ = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=model)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model_ 
    | StrOutputParser()
)

In [21]:
chain.invoke({"question": "What about the Russia-Ukraine war?"})

'The Russia-Ukraine war has seen powerful guided "glide bombs" assisting Russia in rapidly destroying towns, leading to a sense of hopelessness among soldiers. The conflict continues to cause significant devastation in Eastern Ukraine, with towns facing near-total destruction. The situation remains dire as both military actions and humanitarian crises unfold in the region.'

In [22]:
def answer_fn(question, history=None):
    return chain.invoke({"question": question})

In [23]:
from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

CorrectnessMetric evaluation: 100%|██████████| 60/60 [02:04<00:00,  2.07s/it]


In [24]:
display(report)

Loading BokehJS ...

In [25]:
report.to_html("report.html")

# Test Suite 

In [27]:
from giskard.rag import QATestset

testset = QATestset.load("test-set.jsonl")

In [28]:
test_suite = testset.to_test_suite("New York Times Test Suite")

In [33]:
import giskard

def batch_prediction_df(df: pd.DataFrame): 
    return chain.batch([{"question": q} for q in df["question"]])

In [34]:
giskard_model = giskard.Model(
    model=batch_prediction_df,
    model_type="text_generation",
    name="New York Times Question and Answer Model",
    description="This model answers questions about the New York Times website.",
    feature_names=["question"]
)

2024-08-27 13:36:01,471 pid:19128 MainThread giskard.models.automodel INFO     Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.


In [35]:
test_suite_results = test_suite.run(model=giskard_model)

2024-08-27 13:36:03,169 pid:19128 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
2024-08-27 13:36:14,451 pid:19128 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (60, 5) executed in 0:00:11.298689
2024-08-27 13:37:51,494 pid:19128 MainThread root         ERROR    An error happened during test execution for test: TestsetCorrectnessTest
Traceback (most recent call last):
  File "c:\Users\c.manara\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\giskard\core\suite.py", line 702, in run
    result = test_partial.giskard_test(**test_params).execute()
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\c.manara\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\giskard\registry\giskard_test.py", line 195, in execute
    return configured_validate_arguments(self.test_fn)(*self.args, **self.kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
display(test_suite_results)